In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import mode
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:

# Load dataset
df = pd.read_csv("./dataset/heart.csv")

In [3]:

# Encode categorical features
label_encoder = LabelEncoder()
categorical_cols = ['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

# Split the data into features (X) and target variable (y)
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [4]:
# Load pretrained models
ann_model = load_model('./models/ANN.h5')
gru_model = load_model('./models/GRU.h5')
lstm_model = load_model('./models/LSTM.h5')
mlp_model = load_model('./models/MLP.h5')
rnn_model = load_model('./models/SimpleRNN.h5')

In [5]:
# Predict probabilities
y_pred_probs_ann = ann_model.predict(X)
y_pred_probs_gru = gru_model.predict(X[:, None, :])
y_pred_probs_lstm = lstm_model.predict(X[:, None, :])
y_pred_probs_mlp = mlp_model.predict(X)
y_pred_probs_rnn = rnn_model.predict(X[:, None, :])

29/29 [==============================] - 0s 1ms/step


In [6]:

# Convert probabilities to class predictions
y_pred_ann = np.round(y_pred_probs_ann).astype(int)
y_pred_gru = np.round(y_pred_probs_gru).astype(int)
y_pred_lstm = np.round(y_pred_probs_lstm).astype(int)
y_pred_mlp = np.argmax(y_pred_probs_mlp, axis=-1)
y_pred_rnn = np.round(y_pred_probs_rnn).astype(int)

In [7]:
# Ensure predictions are in the same shape
y_pred_ann = y_pred_ann.flatten()
y_pred_gru = y_pred_gru.flatten()
y_pred_lstm = y_pred_lstm.flatten()
y_pred_mlp = y_pred_mlp.flatten()
y_pred_rnn = y_pred_rnn.flatten()


## mode

In [8]:
# Implement voting system using mode
y_pred_combined_mode = np.array([y_pred_ann, y_pred_gru, y_pred_lstm, y_pred_mlp, y_pred_rnn])
y_pred_ensemble_mode = mode(y_pred_combined_mode, axis=0)[0].flatten()

In [9]:
# Evaluate ensemble model
accuracy = accuracy_score(y, y_pred_ensemble_mode)
precision = precision_score(y, y_pred_ensemble_mode)
recall = recall_score(y, y_pred_ensemble_mode)
f1 = f1_score(y, y_pred_ensemble_mode)
roc_auc = roc_auc_score(y, y_pred_ensemble_mode)

In [10]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC AUC Score: {roc_auc}")

Accuracy: 0.9379084967320261
Precision: 0.9430255402750491
Recall: 0.9448818897637795
F1 Score: 0.943952802359882
ROC AUC Score: 0.9370750912233533


## mean

In [11]:
# Implement mean ensemble
y_pred_combined_mean = np.array([y_pred_ann, y_pred_gru, y_pred_lstm, y_pred_mlp, y_pred_rnn])
y_pred_ensemble_mean = np.round(np.mean(y_pred_combined_mean, axis=0)).astype(int)


# For classification tasks, convert the mean probabilities to class labels
y_pred_ensemble_mean_class = np.round(y_pred_ensemble_mean).astype(int)
y_pred_ensemble_mean_class= y_pred_ensemble_mean_class.tolist()



In [12]:
# Evaluate ensemble model
accuracy = accuracy_score(y, y_pred_ensemble_mean_class)
precision = precision_score(y, y_pred_ensemble_mean_class, average='weighted')
recall = recall_score(y, y_pred_ensemble_mean_class,average='weighted')
f1 = f1_score(y, y_pred_ensemble_mean_class,average='weighted')
roc_auc = roc_auc_score(y, y_pred_ensemble_mean_class)

In [13]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC AUC Score: {roc_auc}")

Accuracy: 0.9379084967320261
Precision: 0.937895985623119
Recall: 0.9379084967320261
F1 Score: 0.9379011166274744
ROC AUC Score: 0.9370750912233533


In [14]:
ann_accuracy = accuracy_score(y, y_pred_ann)
print("ANN Accuracy:", ann_accuracy)

ANN Accuracy: 0.9400871459694989


In [15]:
gru_accuracy = accuracy_score(y, y_pred_gru)
print("GRU Accuracy:", gru_accuracy)

GRU Accuracy: 0.9215686274509803


In [16]:
lstm_accuracy = accuracy_score(y, y_pred_lstm)
print("LSTM Accuracy:", lstm_accuracy)


LSTM Accuracy: 0.9248366013071896


In [17]:
mlp_accuracy = accuracy_score(y, y_pred_mlp)
print("MLP Accuracy:", mlp_accuracy)

MLP Accuracy: 0.9237472766884531


In [18]:
rnn_accuracy = accuracy_score(y, y_pred_rnn)
print("RNN Accuracy:", rnn_accuracy)

RNN Accuracy: 0.9411764705882353
